## Timescales

Environment

In [1]:
import sys
sys.path.append("/home/zanardi/Codes/ML/RONEK/ronek/")

from ronek import env
env.set(
  device="cpu",
  device_idx=0,
  nb_threads=8,
  floatx="float64"
)

Import libraries

In [2]:
import json
import numpy as np
import scipy as sp

from ronek import utils
from ronek.systems import TASystem
from silx.io.dictdump import h5todict

Define inputs
> Equilibrium parameters

In [3]:
T = 1.5e4
rho = 1e-2

> Paths

In [4]:
path_to_dtb = "/home/zanardi/Codes/ML/RONEK/ronek/examples/RVC_O3/database/"
path_to_bases = {
  m: f"/home/zanardi/Codes/ML/RONEK/run/RVC_O3/test/max_mom_{m}/cobras_bases.hdf5" for m in (2,10)
}
rom_dims = {
  10: 2,
  30: 10
}

Initialize isothermal master equation model

In [5]:
system = TASystem(
  species={
    k: path_to_dtb + f"/species/{k}.json" for k in ("atom", "molecule")
  },
  rates_coeff=path_to_dtb + "/kinetics.hdf5",
  use_einsum=False,
  use_factorial=True,
  use_arrhenius=True
)

Eigedeomposion of the linear operator

In [6]:
A = {}

> FOM

In [7]:
system.update_fom_ops(T)
n_eq = system.mix.compute_eq_comp(rho)
A["FOM"] = system._compute_lin_fom_ops_a_full(n_eq[0], by_mass=False)

> ROM

In [8]:
bases = utils.map_nested_dict(path_to_bases, h5todict)

In [9]:
for (r, m) in rom_dims.items():
  # Set basis
  phi = bases[m]["phi"][:,:r]
  psi = bases[m]["psi"][:,:r]
  system.set_basis(phi, psi)
  # Compute ROM linear operator
  A[f"ROM-{r+1}"] = system._compute_lin_fom_ops_a_full(
    n_a_eq=n_eq[0],
    phi=system.phi,
    psi=system.psi,
    by_mass=False
  )

> Compute fastest timescale

In [10]:
def get_timescale(A):
  l = sp.linalg.eigvals(A)
  lmin = np.abs(l.real.min())
  return 1.0/lmin

In [11]:
timescales = utils.map_nested_dict(A, get_timescale)
with open("./timescales.json", "w") as file:
  json.dump(timescales, file, indent=2)